In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [1]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) **0.5)
#b1 = torch.randn(n_hidden,                        generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.01
b2 = torch.randn(vocab_size,                      generator=g) * 0.01
gain = torch.ones((1, n_hidden))
bias = torch.ones((1, n_hidden))
batch_mean_running = torch.zeros((1, n_hidden))
batch_std_running = torch.zeros((1, n_hidden))

parameters = [C, W1, W2, b2, gain, bias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True 

12097


In [10]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
  emb = C[Xtr[ix]]
  pre_activation = emb.view(emb.shape[0], -1) @ W1 # + b1 not needed since we have a running batch bias
    
  batch_meani = pre_activation.mean(0, keepdim=True)
  batch_stdi = pre_activation.std(0, keepdim=True)
  pre_activation = gain * (pre_activation - batch_meani) / batch_stdi + bias
  with torch.no_grad():
    batch_mean_running = 0.999 * batch_mean_running + 0.001 * batch_meani
    batch_std_runnning = 0.999 * batch_std_running + 0.001 * batch_stdi
      
  h = torch.tanh(pre_activation)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Ytr[ix])
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())


      0/ 200000: 3.3053
  10000/ 200000: 1.9817
  20000/ 200000: 2.6418
  30000/ 200000: 2.0010
  40000/ 200000: 2.3118
  50000/ 200000: 1.8986
  60000/ 200000: 2.0527
  70000/ 200000: 2.3328
  80000/ 200000: 2.4095
  90000/ 200000: 2.2383
 100000/ 200000: 2.2589
 110000/ 200000: 2.2823
 120000/ 200000: 1.6924
 130000/ 200000: 1.9833
 140000/ 200000: 2.2004
 150000/ 200000: 2.0062
 160000/ 200000: 1.9517
 170000/ 200000: 2.4099
 180000/ 200000: 2.1579
 190000/ 200000: 2.2084


In [11]:
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x]
  pre_activation = emb.view(emb.shape[0], -1) @ W1 # + b1 not needed since we have a running batch bias 
  pre_activation = gain * (pre_activation - batch_mean_running) / batch_std_running + bias
  h = torch.tanh(pre_activation)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss("train")
split_loss("val")

train 3.522601366043091
val 3.5618789196014404


In [12]:
#--Pytorchify--

In [3]:
class Linear:
    def __init__(self, fan_in, fan_out, g, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + [self.bias if self.bias else []]

class BatchNorm1D:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.ones(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):

        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhar + self.beta

        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out

class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
        
    def parameters(self):
        return []

                
n_embd = 10
n_hidden_layers = 100
g = torch.Generator().manual_seeed(21357598)

C = torch.randn((vocab, n_embd), generator=g)
layers = [
    Linear(n_embd * block_size, n_embd, g), Tanh(),
    Linear(n_hidden_layers, n_hidden_layers, g), Tanh(),
    Linear(n_hidden_layers, n_hidden_layers, g), Tanh(),
    Linear(n_hidden_layers, n_hidden_layers, g), Tanh(),
    Linear(n_hidden_layers, vocab_size, g),
]

